In [28]:
import sys
import os

# Add the `src` folder to Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [29]:
cd /Users/srujanappu/Text-Summarizer-Project/research

/Users/srujanappu/Text-Summarizer-Project/research


In [30]:
os.chdir("../")


In [31]:
%pwd

'/Users/srujanappu/Text-Summarizer-Project'

In [32]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: List


In [33]:

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
        )

        return data_validation_config

In [38]:
import os

from textSummarizer.logging import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = True
            all_files = os.listdir(os.path.join("artifacts", "data_ingestion", "summarizer-data","samsum_dataset"))

            for file in self.config.ALL_REQUIRED_FILES:
                if file not in all_files:
                    validation_status = False
                    logger.warning(f"Missing file: {file}")
                    break

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            logger.info(f"Validation status written to {self.config.STATUS_FILE}")
            return validation_status

        except Exception as e:
            logger.exception("Error during data validation")
            raise e


In [39]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    
    data_validation.validate_all_files_exist()

except Exception as e:
    raise e


[2025-03-27 07:41:57,372: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-03-27 07:41:57,373: INFO: common: YAML file: params.yaml loaded successfully]
[2025-03-27 07:41:57,374: INFO: common: Created directory at: artifacts]
[2025-03-27 07:41:57,375: INFO: common: Created directory at: artifacts/data_validation]
[2025-03-27 07:41:57,375: INFO: 1604618475: Validation status written to artifacts/data_validation/status.txt]
